In [0]:
import pandas as pd
import random
import keras
from keras.models import Sequential
from keras.layers import *
from keras import backend as k
from keras import optimizers
from keras.models import Model
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
from time import time
from keras.models import load_model
from google.colab import files
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, normalized_mutual_info_score

In [2]:
uploaded = files.upload()

Saving IR_data.dat to IR_data (1).dat


KeyboardInterrupt: ignored

In [0]:
data = np.load('IR_data.dat')

In [4]:
data.shape

(53, 1500, 25)

In [0]:
features_dim = 1500
wide = 25
BATCH_SIZE  = 4
N1 = 32
N2 = 32
Nfc = 520
L1 = 30
L2 = 15

In [0]:
batches = int(len(data)/BATCH_SIZE)

In [0]:
def batch_generator():
    
  while True:
    
    for batch in range(batches):
      x_ = []
      
      data_ = data[batch * BATCH_SIZE: (batch + 1) * BATCH_SIZE]
      for i in data_:
          x_.append(np.reshape(i, (features_dim, wide)))

      x = np.asarray(x_)
      
      yield (x, x)

In [0]:
#ENCODER

input1 = Input(shape = (features_dim, wide))

encoder = Conv1D(N1, L1, strides = 1, padding = 'SAME', use_bias = True, 
                 bias_initializer = keras.initializers.Constant(value=0.1), activation = 'relu')(input1)

encoder = MaxPooling1D(pool_size = 2, strides = 2)(encoder)

encoder = Conv1D(N2, L2, strides = 1, padding = 'SAME', use_bias = True, 
                 bias_initializer = keras.initializers.Constant(value=0.1), activation = 'relu')(encoder)

encoder = MaxPooling1D(pool_size = 2, strides = 2)(encoder)

out_shape = list(k.int_shape(encoder))

encoder = Flatten()(encoder)

encoder = Dense(Nfc, activation = 'relu')(encoder)

encoder = Dropout(0.5)(encoder)

encoder = Dense(20, activation = 'sigmoid')(encoder)


#DECODER

decoder = Dense((out_shape[1] * out_shape[2]), activation = 'relu')(encoder)

decoder = Dropout(0.5)(decoder)

decoder = Reshape((out_shape[1], out_shape[2]))(decoder)

decoder = UpSampling1D(size = 2)(decoder)

decoder = Conv1D(N2, L2, strides = 1, padding = 'SAME', use_bias = True, 
                 bias_initializer = keras.initializers.Constant(value=0.1), activation = 'relu')(decoder)

decoder = UpSampling1D(size = 2)(decoder)

decoder = Conv1D(N1, L1, strides = 1, padding = 'SAME', use_bias = True, 
                 bias_initializer = keras.initializers.Constant(value=0.1), activation = 'relu')(decoder)

out = Conv1D(wide, 1, padding = 'same', activation = 'sigmoid')(decoder)


autoencoder = Model(input1, out)

In [9]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1500, 25)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1500, 32)          24032     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 750, 32)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 750, 32)           15392     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 375, 32)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 520)               6240520   
__________

In [0]:
encoder_ = Model(input1, encoder)

In [11]:
encoder_.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1500, 25)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1500, 32)          24032     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 750, 32)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 750, 32)           15392     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 375, 32)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 520)               6240520   
__________

In [0]:
adam_ = optimizers.Adam(lr = 1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-8)

In [0]:
def root_mean_squared_error(y_true, y_pred):
  return k.sqrt(k.mean(k.square(y_pred - y_true), axis=-1))

In [0]:
autoencoder.compile(loss = root_mean_squared_error, optimizer = adam_)

In [24]:
autoencoder.fit_generator(batch_generator(), steps_per_epoch = batches, epochs = 5, shuffle = True)

Epoch 1/5
13/13 [==============================] - 7s 514ms/step - loss: 22.9996
Epoch 2/5
13/13 [==============================] - 3s 240ms/step - loss: 22.9876
Epoch 3/5
13/13 [==============================] - 3s 240ms/step - loss: 22.9760
Epoch 4/5
13/13 [==============================] - 3s 240ms/step - loss: 22.9631
Epoch 5/5
13/13 [==============================] - 3s 238ms/step - loss: 22.9493


In [0]:
km = KMeans(n_jobs=-1, n_clusters=10, n_init=20)

In [0]:
encode_pred = encoder_.predict(data)